$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
$$
# Part 1: Backpropagation
<a id=part1></a>

In this part we will learn about **backpropagation** and **automatic differentiation**. We'll implement both of these concepts from scratch and compare our implementation to `PyTorch`'s built in implementation (`autograd`).

In [2]:
import torch
import unittest

%load_ext autoreload
%autoreload 2

test = unittest.TestCase()

The backpropagation algorithm is at the core of training deep models. To state the problem we'll tackle in this notebook, imagine we have an L-layer MLP model, defined as
$$
\hat{\vec{y}^i} = \vec{y}_L^i= \varphi_L \left(
\mat{W}_L \varphi_{L-1} \left( \cdots
\varphi_1 \left( \mat{W}_1 \vec{x}^i + \vec{b}_1 \right)
\cdots \right)
+ \vec{b}_L \right),
$$

a pointwise loss function $\ell(\vec{y}, \hat{\vec{y}})$ and an empirical loss over our entire data set,
$$
L(\vec{\theta}) = \frac{1}{N} \sum_{i=1}^{N} \ell(\vec{y}^i, \hat{\vec{y}^i}) + R(\vec{\theta})
$$

where $\vec{\theta}$ is a vector containing all network parameters, e.g.
$\vec{\theta} = \left[ \mat{W}_{1,:}, \vec{b}_1, \dots,  \mat{W}_{L,:}, \vec{b}_L \right]$.

In order to train our model we would like to calculate the derivative
(or **gradient**, in the multivariate case) of the loss with respect to each and every one of the parameters,
i.e. $\pderiv{L}{\mat{W}_j}$ and $\pderiv{L}{\vec{b}_j}$ for all $j$.
Since the gradient "points" to the direction of functional increase, the negative gradient is often used as a descent direction for descent-based optimization algorithms.
In other words, iteratively updating each parameter proportianally to it's negetive gradient can lead to
convergence to a local minimum of the loss function.

Calculus tells us that as long as we know the derivatives of all the functions "along the way"
($\varphi_i(\cdot),\ \ell(\cdot,\cdot),\ R(\cdot)$)
we can use the **chain rule** to calculate the derivative 
of the loss with respect to any one of the parameter vectors.
Note that if the loss $L(\vec{\theta})$ is scalar (which is usually the case), the gradient of a parameter
will have the same shape as the parameter itself (matrix/vector/tensor of same dimensions).

For deep models that are a composition of many functions, calculating the gradient of each parameter by hand and implementing hard-coded gradient derivations quickly becomes infeasible.
Additionally, such code makes models hard to change, since any change potentially requires re-derivation and re-implementation of the entire gradient function.

The backpropagation algorithm, which we saw [in the lecture](https://vistalab-technion.github.io/cs236605/lecture_notes/lecture_3/#error-backpropagation), provides us with a effective method of applying the **chain rule** recursively so that we can implement gradient calculations of arbitrarily deep or complex models.

We'll now implement backpropagation using a modular software design which will allow us to chain many components layers together and get automatic gradient calculation of the output with respect to the input or any intermediate parameter.

To do this, we'll define a (very poorly named) class: `Block`. Here's the API of this class:

In [3]:
import hw2.blocks as blocks
help(blocks.Block)

Help on class Block in module hw2.blocks:

class Block(abc.ABC)
 |  A block is some computation element in a network architecture which
 |  supports automatic differentiation using forward and backward functions.
 |  
 |  Method resolution order:
 |      Block
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  backward(self, dout)
 |      Computes the backward pass of the block, i.e. the gradient
 |      calculation of the final network output with respect to each of the
 |      parameters of the forward function.
 |      :param dout: The gradient of the network with respect to the
 |      output of this block.
 |      :return: A tuple with the same number of elements as the parameters of
 |      the forward function. Each element will be the gradient of the
 |      network output with respe

In other words, a `Block` can be anything: a layer, an activation function, a loss function or generally *any computation that we know how to derive a gradient for*.

Each block must define a `forward()` function and a `backward()` function.
- The `forward()` function performs the actual calculation/operation of the block and returns an output.
- The `backward()` function computes the gradient of the input and parameters as a function of the gradient of the **output**, according to the chain rule.

This may sound confusing, so here's a diagram illustrating what a `Block` does:

<img src="imgs/backprop.png" width="900" />

Note that the diagram doesn't show that if $f(\vec{x},\vec{y})$ is parametrized, there are also gradients to calculate for the parameters.

The forward pass is pretty straightforward: just do the computation.
To understand the backward pass, imagine that there's some "downstream" loss function
$L(\vec{\theta})$ and magically somehow we are told the gradient of that loss with respect
to the output of our block, $\pderiv{L}{\vec{z}}$.

Now, since $f(\vec{x},\vec{y})$ is a function we know how to derivate,
it means we know how to calculate $\pderiv{\vec{z}}{\vec{x}}$, $\pderiv{\vec{z}}{\vec{y}}$ and also the derivatives with respect to any parameters that $f$ uses.
Thanks to the chain rule, this is all we need to calculate the gradients of the **loss** w.r.t. the input and
parameters.

## Comparison with PyTorch
<a id=part1_1></a>

PyTorch has the `nn.Module` base class, which is similar to our `Block` since it also represents a computation element in a network.
However PyTorch's `nn.Module`s don't compute the gradient,
only perform the forward calculations.
Instead, the `autograd` module tracks operations on tensors, creating a graph of operations, each with it's own `backward` function. Therefore, in PyTorch the `backward()` function is called on the tensors, not the modules.

Here we'll implement a "poor man's autograd": We'll use PyTorch tensors,
but we'll calculate and store the gradients in our blocks (or return them). The gradients we'll calculate are of the entire block, not individual operations on tensors.

To test our  implementation, we'll use PyTorch's `autograd`.

Note that of course this method of tracking gradients is **much** more limited than what PyTorch offers. However it allows us to implement the backpropagation algorithm very simply and really see how it works.

Let's set up some testing instrumentation:

In [4]:
from hw2.grad_compare import compare_block_to_torch

def test_block_grad(block: blocks.Block, x, y=None, delta=1e-2):
    diffs = compare_block_to_torch(block, x, y)
    
    # Assert diff values
    for diff in diffs:
        test.assertLess(diff, delta)

# Show the compare function
compare_block_to_torch??

Notes:
- After you complete your implementation, you should make sure to read and understand the `compare_block_to_torch()` function. It will help you understand what PyTorch is doing.
- The value of `delta` above is somewhat arbitrary. A correct implementation will give you a `diff` of zero.

## Block Implementations
<a id=part1_2></a>

We'll now implement some `Block`s that will enable us to later build an MLP model of arbitrary depth, complete with automatic differentiation.

For each block, you'll first implement the `forward()` function.
Then, you will calculate the derivative of the block by hand with respect to each of its
input tensors and each of its parameter tensors (if any).
Using your manually-calculated derivation, you can then implement the `backward()` function.

### Linear (fully connected) layer

First, we'll implement an affine transform layer, also known as a fully connected layer.

Given an input $\vec{x}$ the layer computes,

$$
\vec{z} = \vec{x} \mattr{W}  + \vec{b} ,~
\mat{W}\in\set{R}^{D_{\mathrm{out}}\times D_{\mathrm{in}}},~ \vec{b}\in\set{R}^{D_{\mathrm{out}}}.
$$

Notes:
- We write it this way to follow the implementation conventions.
- In the code, the input $\vec{x}$ will always be a tensor containing a batch dimension first. Thanks to broadcasting, $\vec{b}$ can remain a vector even if the input is a matrix.

**TODO**: Complete the implementation of the `Linear` class in the `hw2/blocks.py` module.

In [5]:
N = 100
in_features = 200
num_classes = 10

In [6]:
# Test Linear
fc = blocks.Linear(in_features, 1000)
params = fc.params()
x_test = torch.randn(N, in_features)

# Test forward pass
z = fc(x_test)
test.assertSequenceEqual(z.shape, [N, 1000])

# Test backward pass
test_block_grad(fc, x_test)

Comparing gradients... 
input    diff=0.000
param#01 diff=0.000
param#02 diff=0.000


### Activation functions

#### ReLU

ReLU, or rectified linear unit is a very common activation function in deep learning architectures.
In it's most standard form, as we'll implement here, it has no parameters.

The ReLU operation is defined as

$$
\mathrm{relu}(\vec{x}) = \max(\vec{0},\vec{x})
$$

Note that it's not strictly differentiable, however it has sub-gradients. The gradients w.r.t. $\vec{x}$ are simply $1$ for any positive-valued elements and zero for negative-valued elements.

**TODO**: Complete the implementation of the `ReLU` class in the `hw2/blocks.py` module.

In [7]:
# Test ReLU
relu = blocks.ReLU()
x_test = torch.randn(N, in_features)

# Test forward pass
z = relu(x_test)
test.assertSequenceEqual(z.shape, x_test.shape)

# Test backward pass
test_block_grad(relu, x_test)

Comparing gradients... 
input    diff=0.000


C:\Users\Gilad\PycharmProjects\DL2\hw2\blocks.py:158: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return dout * torch.tensor(r > 0, dtype=torch.float)


#### Sigmoid

The sigmoid function $\sigma(x)$ is also sometimes used as an activation function.
We have also seen it previously in the context of logistic regression.

The sigmoid function is defined as

$$
\sigma(\vec{x}) = \frac{1}{1+\exp(-\vec{x})}.
$$

In [8]:
# Test Sigmoid
sigmoid = blocks.Sigmoid()
x_test = torch.randn(N, in_features)

# Test forward pass
z = sigmoid(x_test)
test.assertSequenceEqual(z.shape, x_test.shape)

# Test backward pass
test_block_grad(sigmoid, x_test)

Comparing gradients... 
input    diff=0.000


### Cross-Entropy Loss

As you know by know, this is a common loss function for classification tasks.
In class, we defined it as 

$$\ell_{\mathrm{CE}}(\vec{y},\hat{\vec{y}}) = - {\vectr{y}} \log(\hat{\vec{y}})$$

where $\hat{\vec{y}} = \mathrm{softmax}(x)$ is a probability vector (the output of softmax on the class scores $\vec{x}$) and the vector $\vec{y}$ is a 1-hot encoded class label.

However, it's tricky to compute the gradient of softmax, so instead we'll define a version of cross-entropy that produces the exact same output but works directly on the class scores $\vec{x}$.

We can write,
$$\begin{align}
\ell_{\mathrm{CE}}(\vec{y},\hat{\vec{y}}) &= - {\vectr{y}} \log(\hat{\vec{y}}) 
= - {\vectr{y}} \log\left(\mathrm{softmax}(\vec{x})\right) \\
&= - {\vectr{y}} \log\left(\frac{e^{\vec{x}}}{\sum_k e^{x_k}}\right) \\
&= - \log\left(\frac{e^{x_y}}{\sum_k e^{x_k}}\right) \\
&= - \left(\log\left(e^{x_y}\right) - \log\left(\sum_k e^{x_k}\right)\right)\\
&= - x_y + \log\left(\sum_k e^{x_k}\right)
\end{align}$$

Where the scalar $y$ is the correct class label, so $x_y$ is the correct class score.

Note that this version of cross entroy is also what's [provided](https://pytorch.org/docs/stable/nn.html#crossentropyloss) by PyTorch's `nn` module.

**TODO**: Complete the implementation of the `CrossEntropyLoss` class in the `hw2/blocks.py` module.

In [9]:
# Test CrossEntropy
cross_entropy = blocks.CrossEntropyLoss()
scores = torch.randn(N, num_classes)
labels = torch.randint(low=0, high=num_classes, size=(N,), dtype=torch.long)

# Test forward pass
loss = cross_entropy(scores, labels)
expected_loss = torch.nn.functional.cross_entropy(scores, labels)
test.assertLess(torch.abs(expected_loss-loss).item(), 1e-5)
print('loss=', loss.item())

# Test backward pass
test_block_grad(cross_entropy, scores, y=labels)

loss= 2.7283620834350586
Comparing gradients... 
input    diff=0.000


## Building Models
<a id=part1_3></a>

Now that we have some building `Block`s, we can build an MLP model of arbitrary depth and compute end-to-end gradients.

First, lets copy an idea from PyTorch and implement our own version of the `nn.Sequential` `Module`.
This is a `Block` which contains other `Block`s and calls them in sequence. We'll use this to build our MLP model.

**TODO**: Complete the implementation of the `Sequential` class in the `hw2/blocks.py` module.

In [10]:
# Test Sequential
# Let's create a long sequence of blocks and see
# whether we can compute end-to-end gradients of the whole thing.
# Show the compare function
compare_block_to_torch??

seq = blocks.Sequential(
    blocks.Linear(in_features, 100),
    blocks.Linear(100, 200),
    blocks.Linear(200, 100),
    blocks.ReLU(),
    blocks.Linear(100, 500),
    blocks.Linear(500, 200),
    blocks.ReLU(),
    blocks.Linear(200, 500),
    blocks.ReLU(),
    blocks.Linear(500, 1),
    blocks.Sigmoid(),
)
x_test = torch.randn(N, in_features)

# Test forward pass
z = seq(x_test)
test.assertSequenceEqual(z.shape, [N, 1])

# Test backward pass
test_block_grad(seq, x_test)

Comparing gradients... 
input    diff=0.000
param#01 diff=0.000
param#02 diff=0.000
param#03 diff=0.000
param#04 diff=0.000
param#05 diff=0.000
param#06 diff=0.000
param#07 diff=0.000
param#08 diff=0.000
param#09 diff=0.000
param#10 diff=0.000
param#11 diff=0.000
param#12 diff=0.000
param#13 diff=0.000
param#14 diff=0.000


Now, equipped with a `Sequential`, all we have to do is create an MLP architecture.
We'll define our MLP with the following hyperparameters:
- Number of input features, $D$.
- Number of output classes, $C$.
- Sizes of hidden layers, $h_1,\dots,h_L$.

So the architecture will be:

FC($D$, $h_1$) $\rightarrow$ ReLU $\rightarrow$
FC($h_1$, $h_2$) $\rightarrow$ ReLU $\rightarrow$
$\cdots$ $\rightarrow$
FC($h_{L-1}$, $h_L$) $\rightarrow$ ReLU $\rightarrow$
FC($h_{L}$, $C$)

We'll also create a sequence of the above MLP and a cross-entropy loss, since it's the gradient of the loss that we need in order to train a model.

**TODO**: Complete the implementation of the `MLP` class in the `hw2/models.py` module. Ignore the `dropout` parameter for now.

In [11]:
import hw2.models as models

# Create MLP model
mlp = models.MLP(in_features, num_classes, hidden_features=[100, 50, 100])
print(mlp)


MLP, Sequential
	[0] Linear(200, 100)
	[1] ReLU
	[2] Linear(100, 50)
	[3] ReLU
	[4] Linear(50, 100)
	[5] ReLU
	[6] Linear(100, 10)



In [15]:
# Test MLP architecture
N = 100
in_features = 10
num_classes = 10
for activation in ('relu', 'sigmoid'):
    mlp = models.MLP(in_features, num_classes, hidden_features=[100, 50, 100], activation=activation)
    test.assertEqual(len(mlp.sequence), 7)
    
    num_linear = 0
    for b1, b2 in zip(mlp.sequence, mlp.sequence[1:]):
        if (str(b2).lower() == activation):
            test.assertTrue(str(b1).startswith('Linear'))
            num_linear += 1
            
    test.assertTrue(str(mlp.sequence[-1]).startswith('Linear'))
    test.assertEqual(num_linear, 3)

    # Test MLP gradients
    # Test forward pass
    x_test = torch.randn(N, in_features)
    labels = torch.randint(low=0, high=num_classes, size=(N,), dtype=torch.long)
    z = mlp(x_test)
    test.assertSequenceEqual(z.shape, [N, num_classes])

    # Create a sequence of MLPs and CE loss
    # Note: deliberately using the same MLP instance multiple times to create a recurrence.
    seq_mlp = blocks.Sequential(mlp, mlp, mlp, blocks.CrossEntropyLoss())
    loss = seq_mlp(x_test, y=labels)
    test.assertEqual(loss.dim(), 0)
    print(f'MLP loss={loss}, activation={activation}')

    # Test backward pass
    test_block_grad(seq_mlp, x_test, y=labels)

MLP loss=2.3119688034057617, activation=relu
Comparing gradients... 
input    diff=0.000
param#01 diff=0.010
param#02 diff=0.026
param#03 diff=0.026
param#04 diff=0.035
param#05 diff=0.061
param#06 diff=0.080
param#07 diff=0.104
param#08 diff=0.131
param#09 diff=0.010
param#10 diff=0.026
param#11 diff=0.026
param#12 diff=0.035
param#13 diff=0.061
param#14 diff=0.080
param#15 diff=0.104
param#16 diff=0.131
param#17 diff=0.010
param#18 diff=0.026
param#19 diff=0.026
param#20 diff=0.035
param#21 diff=0.061
param#22 diff=0.080
param#23 diff=0.104
param#24 diff=0.131


C:\Users\Gilad\PycharmProjects\DL2\hw2\blocks.py:158: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return dout * torch.tensor(r > 0, dtype=torch.float)


AssertionError: tensor(0.0263) not less than 0.01

If the above tests passed then congratulations - you've now implemented an arbitrarily deep model and loss function with end-to-end automatic differentiation!